In [1]:
import numpy as np
# import scipy.optimize as opt
# import sys, os, random, gzip
import os
import tensorflow as tf
# from keras import backend as K
# from keras.models import *
# from keras.layers import *
# from keras.optimizers import Adam
# from keras.losses import categorical_crossentropy
import keras
from keras import metrics
# from keras.utils import np_utils
# import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=12000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

# Data 

In [67]:
import tensorflow_datasets as tfds

# Construct a tf.data.Dataset
ds = tfds.load('mnist', split='train', shuffle_files=True)

# Build your input pipeline
# ds = ds.shuffle(1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
ds = ds.shuffle(1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# x = []
# y = []
# for i in range(10):
for example in ds.take(i):
    image, label = example["image"], example["label"]
#         x.append(image)
#         y.append(label)
    

In [43]:
for example in ds.take(3):
    image, label = example["image"], example["label"]

In [68]:
x_train = image
y_train = label

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Custom Loss Function

In [4]:
class MeanSquaredError(tf.keras.losses.Loss):
    
    def call(self, y_true, y_pred):
        y_pred = tf.convert_to_tensor_v2(y_pred)
        y_true = tf.cast(y_true, y_pred.dtype)
        return tf.reduce_mean(math_ops.square(y_pred - y_true), axis=-1)

## Custom Layer

In [ ]:
class KQV(tf.keras.layers.Layer):
    def __init__(self, units=128):
        super(KQV, self).__init__()
        self.units = units
        
    def build(self, input_shape):  # Create the state of the layer (weights)
        wq_init = tf.random_normal_initializer()
        wk_init = tf.random_normal_initializer()
        wv_init = tf.random_normal_initializer()
        
        self.wq = tf.Variable(initial_value=wq_init(shape=(self.units, input_shape[-2] ), dtype='float32'), trainable=True)
        self.wk = tf.Variable(initial_value=wk_init(shape=(self.units, input_shape[-2] ), dtype='float32'), trainable=True)
        self.wv = tf.Variable(initial_value=wv_init(shape=(self.units, input_shape[-2] ), dtype='float32'), trainable=True)

        
#         b_init = tf.zeros_initializer()
#         self.b = tf.Variable(initial_value=b_init(shape=(self.units,), dtype='float32'), trainable=True)
        
    def call(self, inputs):  # Defines the computation from inputs to outputs
        q = tf.matmul(self.wq,inputs)
        k = tf.matmul(self.wk,inputs)
        v = tf.matmul(self.wv, inputs)

        return k, q, v


In [3]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        assert d_model % self.num_heads == 0
        
        self.depth = d_model // self.num_heads
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        
        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
        
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)
        
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)
        
        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
        return output, attention_weights
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()
        
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)
        
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask):
        
        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
        
        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
        
        return out2

In [3]:
class Symmetry_Set_Basis(tf.keras.layers.Layer):
    def __init__(self, node=7, num_out=1):
        super(Symmetry_Set_Basis, self).__init__()
        self.node = node
        self.wq = tf.keras.layers.Dense(node)
        self.wq2 = tf.keras.layers.Dense(node)

        self.wk = tf.keras.layers.Dense(node)
        self.num_out = num_out
    
    def Tile_reshape(self, cn):
        a = cn.shape
#         a = tf.constant(a)
        b = tf.zeros(tf.rank(x))+1
        b = tf.cast(b,tf.int32)
        a = tf.concat([b[:-1],b[-1:]*tf.constant(a[-1], tf.int32)], -1)
        return a
    def VP(self, m, cn): # m: order,  cn: input tensor, k: range
#         cn = tf.cast(cn, tf.float64)
        vp = tf.math.pow(cn,m)
        vp = tf.reduce_sum(vp, axis = -1)
        vp = tf.expand_dims(vp, axis = -1)
        vp = tf.tile(vp, self.Tile_reshape(cn))
        return vp
    
    def VC1(self, cn):
        
        vc = tf.reduce_sum(cn, axis = -1)
        vc = tf.expand_dims(vc, axis=-1)
        vc = tf.tile(vc, self.Tile_reshape(cn))
#         vc = tf.cast(vc, tf.float64)
        return vc
    def VC2(self, cn):
        vc = (self.VC1(cn)**2 - self.VP(2, cn))/2
        return vc
    def VC3(self, cn):
        vc1 = self.VC1(cn)
        vp2 = self.VP(2,cn)
        vp3 = self.VP(3,cn)
        vc = (vc1**3-vp3-3*(vp2 * vc1-vp3 ))/6
        return vc
    def VC4(self, cn):
        vc = (self.VC3(cn)*self.VP(1,cn) - self.VC2(cn)*self.VP(2,cn) + self.VC1(cn)*self.VP(3,cn) - self.VP(4,cn) )/4
        return vc
    
    def call(self, q, k, v):
#         out_num = self.order*2
#         v = tf.expand_dims(v, axis = -1)
#         v = tf.tile(v, self.Tile_reshape(out_num))
        vc1 = self.VC1(v)
        vc2 = self.VC2(v)
        vc3 = self.VC3(v)
        vc4 = self.VC4(v)
        vp2 = self.VP(2,v)
        vp3 = self.VP(3,v)
        vp4 = self.VP(4,v)
        
        vc1 = tf.expand_dims(vc1, axis=-1)
        vc2 = tf.expand_dims(vc2, axis=-1)
        vc3 = tf.expand_dims(vc3, axis=-1)
        vc4 = tf.expand_dims(vc4, axis=-1)
        vp2 = tf.expand_dims(vp2, axis=-1)
        vp3 = tf.expand_dims(vp3, axis=-1)
        vp4 = tf.expand_dims(vp4, axis=-1)

        v = tf.concat([vc1, vc2, vc3, vc4, vp2, vp3, vp4], axis =-1)
#         print("v shape:", v.shape)
        

        q = tf.expand_dims(q, axis=-1)
#         print("q shape:", q.shape)

        q = self.wq(q)
#         print("q shape:", q.shape)
        q = tf.transpose(q, perm=[0, 2, 1]) 
#         print("q shape:", q.shape)

        k = self.wk(v)
#         print("k shape:", k.shape)

        k = tf.transpose(k, perm=[0, 2, 1]) 
#         print("k shape:", k.shape)
#         n = tf.matmul(q,k) ##tooooooooooo slow
        k = k/tf.expand_dims(tf.reduce_max(k,axis=-1), axis=-1)
        q = q/tf.expand_dims(tf.reduce_max(q,axis=-1), axis=-1)
        n = k*q
#         print("n shape:", n.shape)

#         print("v shape:", v.shape)
        n = tf.reduce_sum(n, axis=-1)

#         print("n shape:", n.shape)

        
        v = tf.reduce_max(v, axis=-2)
#         print("v shape:", v.shape)

        pn = tf.math.top_k(n, k = self.num_out)
        n = pn.values
        index = pn.indices
        v = tf.gather(v,index, batch_dims=-1)



#         print("v shape:", v.shape)
        return v
        


        
        


In [22]:
class Operator_Basis(tf.keras.layers.Layer):
    def __init__(self, node=4, num_out=1):
        super(Operator_Basis, self).__init__()
        self.node = node
        self.wq = tf.keras.layers.Dense(node)
        self.wq2 = tf.keras.layers.Dense(node)
        self.wk = tf.keras.layers.Dense(node)
        self.alpha = tf.keras.layers.Dense(1)
        self.num_out = num_out
        
    
    def call(self, q, k, v):
        sqrt = tf.math.sqrt(tf.math.abs(v))
        ln = tf.math.log(v)
        exp = tf.math.exp(v)
        rgsn = self.alpha(tf.expand_dims(v, axis=-1))
        
        sqrt= tf.expand_dims(sqrt, axis=-1)
        ln = tf.expand_dims(ln, axis=-1)
        exp = tf.expand_dims(exp, axis=-1)
#         sqrt = tf.expand_dims(sqrt, axis=-1)

        v = tf.concat([sqrt, ln, exp, rgsn], axis =-1)
        q = tf.expand_dims(q, axis=-1)
        q = self.wq(q)
        q = tf.transpose(q, perm=[0, 2, 1]) 
        k = self.wk(v)
        k = tf.transpose(k, perm=[0, 2, 1]) 
        k = k/tf.expand_dims(tf.reduce_max(k,axis=-1), axis=-1)
        q = q/tf.expand_dims(tf.reduce_max(q,axis=-1), axis=-1)
        n = k*q
        n = tf.reduce_sum(n, axis=-1)
        v = tf.reduce_max(v, axis=-2)
        pn = tf.math.top_k(n, k = self.num_out)
        n = pn.values
        index = pn.indices
        v = tf.gather(v,index, batch_dims=-1)
        return v
        


        
        


In [170]:
tf.zeros(tf.rank(x))+1

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>

In [149]:
tf.zeros()+1

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>

In [22]:
x_train.shape

(60000, 28, 28)

In [20]:
np.sum(x_train, axis = -1).shape

(60000, 28)

In [262]:
d = np.sum(np.sum(x_train, axis = -1), axis=-1)
d=np.expand_dims(d, axis=-1)
d=np.expand_dims(d, axis=-1)

x_train = x_train/d

In [35]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [ ]:
# x_(t+1) = x_t - lr*grad.(f(x_t))

opt = tf.keras.optimizers.Adam(learning_rate=0.1)
var1 = tf.Variable(10.0)
loss = lambda: (var1 ** 2)/2.0       # d(loss)/d(var1) == var1
step_count = opt.minimize(loss, [var1]).numpy()
# The first step is `-learning_rate*sign(grad)`
var1.numpy()

In [28]:
tf.keras.backend.set_floatx('float32')
inputs = tf.keras.Input(shape=(28,28))
x = inputs
# x = tf.cast(x, tf.float64)
print(x.shape)
x = tf.keras.layers.Flatten()(x)

print(x.shape)

x = Symmetry_Set_Basis(num_out=3)(x, x, x)
print(x.shape)
a = Operator_Basis(num_out=2)(x, x, x)
b = Operator_Basis(num_out=2)(x, x, x)
x = Operator_Basis(num_out=2)(x, x, x)
print(x.shape)
x = tf.concat([x,a,b], axis=-1)
print(x.shape)

x = Symmetry_Set_Basis(num_out=3)(x, x, x)
a = Symmetry_Set_Basis(num_out=3)(x, x, x)
b = Symmetry_Set_Basis(num_out=3)(x, x, x)
print(x.shape)
a = Operator_Basis(num_out=1)(a, a, a)
b = Operator_Basis(num_out=1)(b, b, b)
x = Operator_Basis(num_out=1)(x, x, x)
print(x.shape)
x = tf.concat([x,a,b], axis=-1)
print(x.shape)


# print("x shape:")
# x = tf.squeeze(x, axis= -1)
# print(x.shape)
# x = tf.keras.layers.Dense(1)(x)
# print(x.shape)
x = tf.keras.layers.Flatten()(x)
print(x.shape)



# x = tf.keras.layers.Dense(256)(x)
# print(x.shape)
# x = tf.keras.layers.Dense(128)(x)
# print(x.shape)
# x = tf.keras.layers.Dense(32)(x)
# print(x.shape)
x = tf.keras.layers.Dense(10)(x)
print(x.shape)
x = tf.keras.layers.Activation("softmax")(x)
print(x.shape)
modelANN = tf.keras.Model(inputs= inputs, outputs=x, name='ANN')

(None, 28, 28)
(None, 784)
(None, 3)
(None, 2)
(None, 6)
(None, 3)
(None, 1)
(None, 3)
(None, 3)
(None, 10)
(None, 10)


In [29]:
model_type = "ANN"
save_dir = './test1/'
model_name = '%s_model_'% model_type 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True)
csv_logger = keras.callbacks.CSVLogger(save_dir+model_type+'.csv')


earlystop = keras.callbacks.EarlyStopping(
                            monitor="val_loss",
                            min_delta=1e-4,
                            patience=3, # 10
                            verbose=1,
                            mode='min', baseline=None, ## 'min' 
                            restore_best_weights=True)
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=2, min_lr=0.00001)
callbacks = [checkpoint, csv_logger,  earlystop ]

In [30]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [31]:
# modelANN.compile(optimizer='adam' , loss=loss_fn, metrics=['accuracy', metrics.AUC(name="auc")])
modelANN.compile(optimizer='adam',
                 loss=loss_fn,
                 metrics=['accuracy'])
modelANN.summary()

Model: "ANN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
flatten_11 (Flatten)            (None, 784)          0           input_7[0][0]                    
__________________________________________________________________________________________________
symmetry__set__basis_18 (Symmet (None, 3)            70          flatten_11[0][0]                 
__________________________________________________________________________________________________
operator__basis_18 (Operator_Ba (None, 2)            30          symmetry__set__basis_18[0][0]    
________________________________________________________________________________________________

In [306]:
x_train.shape, y_train.shape

((60000, 28, 28), (60000,))

In [118]:
y_train.shape


(60000,)

In [53]:
vec = np.zeros([10])

In [54]:
vec

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [69]:
y2[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [144]:
# y = [ for i in y_train]
y2 = []
for i in y_train:
    vec = np.zeros([10])    
    vec[i]=1
    y2.append(vec)
y2=np.array(y2)    
y2.shape


(60000, 10)

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [107]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [32]:

# modelANN.fit(x_train, y2 , callbacks = callbacks, shuffle=True , epochs=400, batch_size=32, verbose=1)
modelANN.fit(x_train, y_train, epochs=4)

Epoch 1/4
1875/1875 [==============================] - 17s 9ms/step - loss: 2.3643 - accuracy: 0.0969
Epoch 2/4
1875/1875 [==============================] - 17s 9ms/step - loss: 2.3643 - accuracy: 0.0969
Epoch 3/4
1875/1875 [==============================] - 17s 9ms/step - loss: 2.3643 - accuracy: 0.0969
Epoch 4/4
1875/1875 [==============================] - 17s 9ms/step - loss: 2.3643 - accuracy: 0.0969


In [ ]:
# Selection Block
class Selection(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(Selection, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        assert d_model % self.num_heads == 0
        
        self.depth = d_model // self.num_heads
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        
#         self.wv = tf.keras.layers.Dense(d_model)
        
        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
        
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)
        
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)
        
        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
        return output, attention_weights


## Optimize

In [13]:
# x_(t+1) = x_t - lr*grad.(f(x_t))

opt = tf.keras.optimizers.Adam(learning_rate=0.1)
var1 = tf.Variable(10.0)
loss = lambda: (var1 ** 2)/2.0       # d(loss)/d(var1) == var1
step_count = opt.minimize(loss, [var1]).numpy()
# The first step is `-learning_rate*sign(grad)`
var1.numpy()

9.9

In [45]:
while var1.numpy()>0.1:
    opt.minimize(loss, [var1]).numpy()
    print(var1.numpy())

0.9540412
0.9305041
0.90742856
0.8848088
0.862639
0.8409134
0.8196262
0.79877156
0.7783437
0.7583367
0.73874485
0.7195623
0.70078325
0.68240196
0.66441256
0.64680934
0.6295866
0.6127385
0.59625936
0.5801435
0.5643853
0.54897904
0.5339191
0.5191999
0.50481594
0.4907616
0.47703144
0.46361995
0.4505217
0.43773136
0.4252435
0.41305286
0.40115413
0.3895421
0.3782116
0.36715743
0.35637453
0.34585783
0.33560234
0.3256031
0.3158552
0.30635378
0.29709405
0.28807122
0.2792806
0.2707176
0.26237753
0.25425592
0.24634825
0.23865008
0.23115706
0.22386485
0.21676919
0.20986587
0.20315073
0.1966197
0.19026873
0.18409383
0.1780911
0.17225665
0.16658668
0.16107745
0.15572527
0.1505265
0.14547755
0.14057492
0.13581514
0.13119482
0.12671058
0.12235916
0.11813731
0.11404186
0.11006968
0.10621771
0.102482945
0.09886242


In [53]:
a = tf.constant([[1,1],[2,2],[3,3]])
a.numpy()

array([[1, 1],
       [2, 2],
       [3, 3]], dtype=int32)

In [55]:
tf.math.pow(a,2)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 1],
       [4, 4],
       [9, 9]], dtype=int32)>

In [57]:
tf.reduce_sum(a, axis=-1)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 4, 6], dtype=int32)>

# 

## Symmetry Variables Function

In [132]:
## Note: m<=n
def find_N(cn):
    a = cn.shape
    a = tf.constant(a)
    n = a[-1:].numpy()
    n = n[0]
    return n

def Tile_reshape(cn):
    a = cn.shape
    a = tf.constant(a)
    b = a*0+1
    a = tf.concat([b[:-1],b[-1:]*tf.constant(a[-1])], -1)
    return a
def VP(m, cn): # m: order,  cn: input tensor, k: range
    cn = tf.cast(cn, tf.float64)
    vp = tf.math.pow(cn,m)
    vp = tf.reduce_sum(vp, axis = -1)
    vp = tf.expand_dims(vp, axis = -1)
    vp = tf.tile(vp, Tile_reshape(cn))
    return vp
 
def VC1(cn):
    vc = tf.reduce_sum(cn, axis = -1)
    vc = tf.expand_dims(vc, axis=-1)
    vc = tf.tile(vc, Tile_reshape(cn))
    vc = tf.cast(vc, tf.float64)
    return vc
def VC2(cn):
    vc = (VC1(cn)**2 - VP(2, cn))/2
    return vc
def VC3(cn):
    vc1 = VC1(cn)
    vp2 = VP(2,cn)
    vp3 = VP(3,cn)
    vc = (vc1**3-vp3-3*(vp2 * vc1-vp3 ))/6
    return vc
def VC4(cn):
    n = find_N(cn)
#     vc = (VC3(cn)*VP(0,cn) - 3/(n-2)*VC2(cn)*VP(1,cn) + 3/(n-2)*2/(n-1)*VC1(cn)*VP(3,cn) - 3/(n-2)*2/(n-1)*VP(4,cn) )/(n-3)
    vc = (VC3(cn)*VP(1,cn) - VC2(cn)*VP(2,cn) + VC1(cn)*VP(3,cn) - VP(4,cn) )/4
    return vc

## VCN: 
# vcn = 0
# for i in range(N):
#     vcn += VC(N-i-1)VP(i+1)*(-1)**i
# vcn = vcn/N

In [130]:
def Con(n,k):
    fc=1
    for i in range(k):
        fc *=(n-i)
        fc /= (i+1)
    return fc
def text(cn):
    vc = (VC2(cn)*VP(1, cn) - VC1(cn)*VP(2,cn) + VP(3,cn))/3
    return vc

In [120]:
Con(5,3)

10.0

In [58]:
VC3(cn).dtype

tf.float64

In [92]:
a = cn.shape
a = tf.constant(a)
n = a[-1:].numpy()
n[0]

5

In [134]:
text(cn)

<tf.Tensor: shape=(3, 5), dtype=float64, numpy=
array([[ 225.,  225.,  225.,  225.,  225.],
       [  10.,   10.,   10.,   10.,   10.],
       [1175., 1175., 1175., 1175., 1175.]])>

In [135]:
VC3(cn)

<tf.Tensor: shape=(3, 5), dtype=float64, numpy=
array([[ 225.,  225.,  225.,  225.,  225.],
       [  10.,   10.,   10.,   10.,   10.],
       [1175., 1175., 1175., 1175., 1175.]])>

In [128]:
cn = tf.constant([[1,2,3,4,5],[1,1,1,1,1],[3,4,5,6,7]])
# cn = tf.constant([[1,2,3,4],[1,1,1,1],[3,4,5,6]])

# cn.numpy()
VC4(cn).numpy()

array([[ 561.5,  561.5,  561.5,  561.5,  561.5],
       [   5. ,    5. ,    5. ,    5. ,    5. ],
       [3616.5, 3616.5, 3616.5, 3616.5, 3616.5]])

In [112]:
N=5
fc=1
for i in range(N):
    fc*=(i+1)
    
a = 0
for i in range(N):
    a += fc/(i+1)

In [113]:
a

274.0

In [52]:
tf.math.multiply(VC3(cn),VC1(cn))

InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a double tensor but is a int32 tensor [Op:Mul]

In [26]:
a = cn.shape
a = tf.constant(a)
b = a*0+1
a = tf.concat([b[:-1],b[-1:]*tf.constant(a[-1])], -1)
a

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>

In [22]:
a[-1:]

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>

In [11]:
tf.tile(cn, [1,2])

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[1, 1, 1, 1],
       [2, 2, 2, 2],
       [3, 3, 3, 3]], dtype=int32)>